In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="A9FeYLNllljYJ9rzI8vK")
project = rf.workspace("mcastiict").project("zibel")
version = project.version(1)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to zibel-1 in yolov5pytorch:: 100%|██████████| 4705/4705 [00:00<00:00, 11002.04it/s]


In [1]:
from glob import glob

img_list = glob('/home/addinedu/dev_ws/data/zibel-1/train/images/*.jpg')
print(len(img_list))

1878


훈련,테스트 데이터 분할

In [2]:
from sklearn.model_selection import train_test_split

train_img_list = glob('/home/addinedu/dev_ws/data/zibel-1/train/images/*.jpg')
val_img_list = glob('/home/addinedu/dev_ws/data/zibel-1/valid/images/*.jpg')
print(len(train_img_list), len(val_img_list))

1878 470


In [3]:
with open('/home/addinedu/dev_ws/data/zibel-1/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

with open('/home/addinedu/dev_ws/data/zibel-1/val.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')


In [5]:
import yaml

with open('/home/addinedu/dev_ws/data/zibel-1/data.yaml', 'r') as f:
    data = yaml.safe_load(f)

print(data)

data['train'] = '/home/addinedu/dev_ws/data/zibel-1/train.txt'
data['val'] = '/home/addinedu/dev_ws/data/zibel-1/val.txt'


with open('/home/addinedu/dev_ws/data/zibel-1/data.yaml', 'w') as f:
    yaml.dump(data,f)

print(data)

{'names': ['Bin', 'Glass', 'Metal', 'Other', 'Plastic'], 'nc': 5, 'roboflow': {'license': 'CC BY 4.0', 'project': 'zibel', 'url': 'https://universe.roboflow.com/mcastiict/zibel/dataset/1', 'version': 1, 'workspace': 'mcastiict'}, 'test': '../test/images', 'train': '/home/addinedu/dev_ws/data/zibel-1/train.txt', 'val': '/home/addinedu/dev_ws/data/zibel-1/val.txt'}
{'names': ['Bin', 'Glass', 'Metal', 'Other', 'Plastic'], 'nc': 5, 'roboflow': {'license': 'CC BY 4.0', 'project': 'zibel', 'url': 'https://universe.roboflow.com/mcastiict/zibel/dataset/1', 'version': 1, 'workspace': 'mcastiict'}, 'test': '../test/images', 'train': '/home/addinedu/dev_ws/data/zibel-1/train.txt', 'val': '/home/addinedu/dev_ws/data/zibel-1/val.txt'}


모델 생성

In [6]:
%cd /home/addinedu/yolo/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /home/addinedu/dev_ws/data/zibel-1/data.yaml --cfg /home/addinedu/yolo/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name recycle_yolov5s_2


/home/addinedu/yolo/yolov5


/home/addinedu/venv/YOLO_venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


train: weights=yolov5s.pt, cfg=/home/addinedu/yolo/yolov5/models/yolov5s.yaml, data=/home/addinedu/dev_ws/data/zibel-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=recycle_yolov5s_2, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-312-g1bcd17ee Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5927MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_de

In [7]:
import torch
torch.cuda.get_device_name(0)	#gpu 확인
torch.cuda.is_available()

True

/home/addinedu/yolo/yolov5/runs/train/trouble_result2/weights/best.pt

생성한 모델을 가지고 테스트 해보기

In [8]:
from IPython.display import Image
import os

# val_img_path = '/home/addinedu/dev_ws/data/asd/trouble/images.jpeg'
val_img_path = val_img_list[1]
%cd /home/addinedu/yolo/yolov5/
!python detect.py --weights /home/addinedu/yolo/yolov5/runs/train/recycle_yolov5s_2/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}"


/home/addinedu/yolo/yolov5


/home/addinedu/venv/YOLO_venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


detect: weights=['/home/addinedu/yolo/yolov5/runs/train/recycle_yolov5s_2/weights/best.pt'], source=/home/addinedu/dev_ws/data/zibel-1/valid/images/62c1715b8961562c1715b8961a_frame18_jpg.rf.8f589a2e5f5899b9a3c6e3555eab7285.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-312-g1bcd17ee Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5927MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /home/addinedu/dev_ws/data/zibel-1/valid/images/62c1715b8961562c1715b8961a_frame18_jpg.rf.8f589a2e5f5899b9a3c6e3555eab7285.jpg: 416x416 1 Plastic, 5.2ms
Spe

텍스트 제거, 라인두께 얇게

In [2]:
import os

val_img_path = '/home/addinedu/dev_ws/data/asd/trouble/maxresdefault.jpg'
yolov5_dir = '/home/addinedu/yolo/yolov5'

os.chdir(yolov5_dir)
command = f"python detect.py --weights /home/addinedu/yolo/yolov5/runs/train/trouble_result2/weights/best.pt --img 416 --conf 0.5 --source '{val_img_path}' --hide-labels --line-thickness 1"
os.system(command)

detect: weights=['/home/addinedu/yolo/yolov5/runs/train/trouble_result5/weights/best.pt'], source=/home/addinedu/dev_ws/data/asd/trouble/maxresdefault.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=True, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5927MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20873139 parameters, 0 gradients, 47.9 GFLOPs
image 1/1 /home/addinedu/dev_ws/data/asd/trouble/maxresdefault.jpg: 256x416 (no detections), 42.1ms
Speed: 1.0ms pre-process, 42.1ms inference, 7.4ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp17


0

In [1]:
%cd /home/addinedu/yolo/yolov5/

!python train.py --img 416 --batch 16 --epochs 20 --data /home/addinedu/dev_ws/data/acne-new.v35i.yolov5pytorch/dataset/data.yaml --cfg /home/addinedu/yolo/yolov5/models/yolov5m.yaml --weights yolov5s.pt --name trouble_result5


/home/addinedu/yolo/yolov5


/home/addinedu/venv/YOLO_venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


train: weights=yolov5s.pt, cfg=/home/addinedu/yolo/yolov5/models/yolov5m.yaml, data=/home/addinedu/dev_ws/data/acne-new.v35i.yolov5pytorch/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=trouble_result5, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2060, 5927MiB)

hyperparameters: lr0=0.01, lrf=0.01,